In [1]:
from keras.applications.mobilenet_v2 import preprocess_input
from keras.preprocessing.image import img_to_array
from keras.models import load_model
import numpy as np
import cv2
import os


In [2]:
prototxtpath=("face_detector/deploy.prototxt")
weightspath=("face_detector/res10_300x300_ssd_iter_140000.caffemodel")
net=cv2.dnn.readNet(prototxtpath,weightspath)
model=load_model("model")`

In [3]:
image=cv2.imread("images/pic2.jpg")
orig=image.copy()
(h,w)=image.shape[:2]

blob=cv2.dnn.blobFromImage(image,1.0,(300,300),(104.0,177.0,123.0))
net.setInput(blob)
detections=net.forward()
#detections.shape
#detections

In [4]:
for i in range(0,detections.shape[2]):
    confidence=detections[0,0,i,2]
    if(confidence > 0.5):
        box=detections[0,0,i,3:7]*np.array([w,h,w,h])
        (startx,starty,endx,endy)=box.astype("int")
        (startx,starty)=(max(0,startx),max(0,starty))
        (endx,endy)=(min(w-1,endx),min(h-1,endy))
        face=image[starty:endy,startx:endx]
        face=cv2.cvtColor(face,cv2.COLOR_BGR2RGB)
        face=cv2.resize(face,(224,224))
        face=img_to_array(face)
        face=preprocess_input(face)
        face=np.expand_dims(face,axis=0)

        (mask,withoutmask)=model.predict(face)[0]
        
        label="Mask" if mask > withoutmask else "No Mask"
        color=(0,255,0) if label == "Mask" else (0,0,255)
        label="{}:{:.2f}%".format(label,max(mask,withoutmask)*100)
        cv2.putText(image,label,(startx,starty-10),cv2.FONT_HERSHEY_SIMPLEX,0.45,color,2)
        cv2.rectangle(image,(startx,starty),(endx,endy),color,2)
        
cv2.imshow("OUTPUT",image)
cv2.waitKey(0)        

-1